<a href="https://colab.research.google.com/github/giovannasantoloni/Projetos-de-Sistemas-Inteligentes-de-Apoio-Decis-o/blob/main/Projeto_Final_Curso_BI_Master_2021_1_Giovanna_Dohnert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Final Curso BI Master 2021-1
> Por Giovanna Santoloni Dohnert

In [ ]:
#Montagem do Drive 
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
#Importar Bibliotecas
import numpy as np
import pandas as pd
import os
import shutil
import cv2
from glob import glob
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras
import random
import shutil
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
#Importar Aplicações
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from scipy import ndimage, misc
import skimage
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Sequential
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models.cloning import training
from tensorflow.keras.layers import Dense, Dropout


In [ ]:
# Caminho da Base de Dados
import os
workdir_path = '/content/drive/My Drive/Projeto Final BI/images' 
os.chdir(workdir_path)


In [ ]:
# Caminho do dataset
full_dataset = r'/content/drive/My Drive/Projeto Final BI/images'

# Definição de Parâmetros:
A definição de parâmetros em um modelo de classificação de imagens é essencial para que o modelo possa aprender a partir dos dados de treinamento e, posteriormente, ser capaz de classificar novas imagens de forma precisa. A escolha adequada dos parâmetros pode melhorar significativamente a precisão do modelo e sua capacidade de generalização para novos dados. 


In [ ]:
#Definição de Parâmetros

#Número de Classes 
n_classes = 9

#Tamanho das Imagens
target_size = (224, 224)

#Tamanho do Lote para Treinamento
batch_size = 32


# Separando o Conjunto de Dados em Treino, Validação e Teste

Separamos o conjunto de dados em treino, validação e teste a fim de avaliar o desempenho do modelo frente a diferentes dados e garantir que ele seja capaz de generalizar de maneira adequada. 

•	Conjunto de Treino: usado para treinar o modelo, ajustar os pesos e os parâmetros do modelo com base nos dados de treinamento.

•	Conjunto de Validação: usado para avaliar o desempenho do modelo em dados que ele não viu durante o treinamento e otimizar os parâmetros do modelo para obter um melhor desempenho.

•	Conjunto de Teste: usado para avaliar o desempenho final do modelo em dados completamente novos que não foram usados para treinamento ou validação.

# Data Augmentation

O data augmentation foi utilizada para aumentar a quantidade de dados de treinamento do modelo, através da geração de novas amostras a partir das amostras existentes, aplicando transformações como rotação, espelhamento (...) entre outros.

In [ ]:
# Separando o Conjunto de Dados em Treino e Validação

image_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2, rotation_range=20, # aumentando a variabilidade de dados
width_shift_range=0.2,
height_shift_range=0.2,
zoom_range=0.2,
horizontal_flip=True
)

train_dataset = image_generator.flow_from_directory(batch_size=batch_size, 
                                                    directory=full_dataset ,
                                                    shuffle = True,
                                                    target_size =target_size,
                                                    subset ="training",
                                                    class_mode = 'categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=batch_size, 
                                                    directory=full_dataset,
                                                    shuffle = True,
                                                    target_size =target_size,
                                                    subset ="validation",
                                                    class_mode = 'categorical')

Found 4239 images belonging to 9 classes.
Found 1055 images belonging to 9 classes.


In [ ]:
# Separando o conjunto de Teste
image_generator = ImageDataGenerator(rescale=1./255)

test_dataset = image_generator.flow_from_directory(batch_size=batch_size, 
                                                    directory=full_dataset,
                                                    shuffle = True,
                                                    target_size =target_size,
                                                    class_mode = 'categorical')

Found 5294 images belonging to 9 classes.


# Utilização do Transfer Learning
O uso de Transfer Learning e pesos pré-treinados do modelo VGG16 ajuda a melhorar a precisão do modelo de classificação de imagens, a reduzir o tempo e o custo de treinamento e a lidar com conjuntos de dados pequenos.


In [ ]:
# carregar a arquitetura VGG16 pré-treinada
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(*target_size, 3))

In [33]:
# adicionar camadas adicionais ao modelo
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x) 

# Adicionamos uma camada de dropout de 0,5, o que significa que metade dos neurônios da camada anterior serão aleatoriamente desativados
# durante o treinamento, para evitar overfitting.

x = Dropout(0.5)(x)  

predictions = Dense(n_classes, activation='softmax')(x)

In [34]:
# criar modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# congelar as camadas da base
for layer in base_model.layers:
    layer.trainable = False

In [35]:
# compilar o modelo
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['acc'])

In [36]:
# Treinar o modelo
history = model.fit(
    train_dataset,
    steps_per_epoch=len(train_dataset),
    epochs=10,
    validation_data=validation_dataset,
    validation_steps=len(validation_dataset))

Epoch 1/10
133/133 [==============================] - 120s 891ms/step - loss: 1.3495 - acc: 0.5414 - val_loss: 1.1446 - val_acc: 0.5858
Epoch 2/10
133/133 [==============================] - 119s 895ms/step - loss: 1.1978 - acc: 0.5891 - val_loss: 1.1398 - val_acc: 0.5791
Epoch 3/10
133/133 [==============================] - 116s 875ms/step - loss: 1.1436 - acc: 0.5971 - val_loss: 1.0847 - val_acc: 0.6095
Epoch 4/10
133/133 [==============================] - 119s 897ms/step - loss: 1.1025 - acc: 0.6112 - val_loss: 1.0632 - val_acc: 0.6218
Epoch 5/10
133/133 [==============================] - 137s 1s/step - loss: 1.0602 - acc: 0.6251 - val_loss: 1.0423 - val_acc: 0.6057
Epoch 6/10
133/133 [==============================] - 119s 892ms/step - loss: 1.0375 - acc: 0.6310 - val_loss: 1.0354 - val_acc: 0.6171
Epoch 7/10
133/133 [==============================] - 115s 868ms/step - loss: 1.0334 - acc: 0.6299 - val_loss: 1.0299 - val_acc: 0.6360
Epoch 8/10
133/133 [==============================]

In [38]:
# Avaliando o modelo com conjunto de teste
y_pred = model.predict(test_dataset)
y_pred = np.argmax(y_pred, axis=1)
y_true = test_dataset.classes

166/166 [==============================] - 37s 220ms/step


In [39]:
# Gerando a Matriz de Confusão
conf_matrix = confusion_matrix(y_true, y_pred)
print("Matriz de Confusão:\n", conf_matrix)

Matriz de Confusão:
 [[  63    0    0   93    0  822   13]
 [   0    0    0    0    0    3    0]
 [  10    0    0   14    0  157    3]
 [  65    0    0   88    0  679    6]
 [   8    0    1   18    0  150    4]
 [ 195    0    0  321    0 2342   30]
 [  16    0    0   21    0  170    2]]


# Conclusões 

O modelo apresentou um desempenho razoável, contudo ainda parece sofrer com overfitting, já que a acurácia no conjunto de validação apresenta-se consideravelmente menor do que a acurácia no conjunto de treinamento. Isso pode ser observado nas diferenças entre as acurácias do conjunto de treinamento e validação nas epochs 1, 2, 3 e 5.

O valor da função de perda (loss) no conjunto de validação também não reduz consistentemente com as epochs, o que sugere que o modelo pode estar estagnado do ponto de vista de aprendizado. Em resumo,  o modelo possa se beneficiará de ajustes nos hiperparâmetros para otimizar seu desempenho.                                                                

A partir da matriz de confusão, podemos observar que o modelo não está performando bem para algumas classes específicas. Por exemplo, para a classe 2, não foi possível prever nenhuma raça corretamente, o que indica que o modelo precisa ser melhorado para esta classe. O modelo também apresentou baixo desempenho para as classes 1, 4 e 7.

Podemos calcular a acurácia do modelo a partir da matriz de confusão, o que aponta para o percentual de corretas em relação ao total de predições: A acurácia é de aproximadamente 53,8%.